In [10]:
#conexao com o banco
import psycopg2

try:
    conn = psycopg2.connect(
        host="localhost",
        database="vendas_db",
        user="postgres",
        password="postgres"
    )
    print("Conexão bem sucedida!")
except Exception as e:
    print("Erro ao conectar ao banco de dados:", e)

Conexão bem sucedida!


In [6]:
#preparar a tabela do csv
import pandas as pd

df = pd.read_csv('vendas_projeto_xpto_csv_final.csv')

df_novo = df[['cod_produto','nome_produto','categoria','valor_venda','venda_final','nome_vendedor','data_venda']]

df_novo.head()


,cod_produto,nome_produto,categoria,valor_venda,venda_final,nome_vendedor,data_venda
0,SKU-0000001,LG K10 TV Power,Celulares,679.00,814.8000,Ana Teixeira,2012-10-04
1,SKU-0000002,Geladeira Duplex,Eletrodomésticos,832.00,956.8000,Josias Silva,2012-01-01
2,SKU-0000003,Lavadora 11 Kg,Eletrodomésticos,790.00,908.5000,Josias Silva,2012-02-02
3,SKU-0000004,Lavadora 11 Kg,Eletrodomésticos,765.32,880.1180,Mateus Gonçalves,2012-03-03
4,SKU-0000005,Lavadora 11 Kg,Eletrodomésticos,459.89,528.8735,Artur Moreira,2012-04-04


In [11]:
#criar a tabela no banco

create_table_query = """
CREATE TABLE IF NOT EXISTS vendas_final (
    id SERIAl PRIMARY KEY,
    cod_produto TEXT,
    nome_produto TEXT,
    categoria TEXT,
    valor_venda NUMERIC,
    venda_final NUMERIC,
    nome_vendedor TEXT,
    data_venda DATE
);
"""

# Criar o cursor
cursor = conn.cursor()

cursor.execute(create_table_query)

conn.commit()

cursor.close()

In [12]:
#inserir os dados do dataframe na tabela
cursor = conn.cursor()

for index, row in df_novo.iterrows():
    cursor.execute("""
        INSERT INTO vendas_final (cod_produto, nome_produto, categoria, valor_venda, venda_final, nome_vendedor, data_venda)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """,(
            row['cod_produto'], 
            row['nome_produto'], 
            row['categoria'], 
            row['valor_venda'], 
            row['venda_final'], 
            row['nome_vendedor'], 
            row['data_venda']
    ))

conn.commit()

cursor.close()

In [16]:
#Consultas SQL
#1. Qual a quantidade de cada produto foi vendida em cada ano?
query= """
    SELECT nome_produto, count(nome_produto),EXTRACT(YEAR FROM data_venda)  FROM vendas_final vf 
    GROUP BY nome_produto,EXTRACT(YEAR FROM data_venda) 
    ORDER BY EXTRACT(YEAR FROM data_venda)
"""

cursor = conn.cursor()
cursor.execute(query)

# Pegar os resultados
resultados = cursor.fetchall()

# Nome das colunas
colunas = ['nome_produto', 'quantidade', 'ano']

# Criar o DataFrame
df_resultado = pd.DataFrame(resultados, columns=colunas)

cursor.close()
# Exibir a tabela
print(df_resultado)

                nome_produto  quantidade   ano
0           Desktop HP 16 GB           2  2012
1          Forno-Micro-Ondas           1  2012
2             Lavadora 11 Kg           4  2012
3           Geladeira Duplex          10  2012
4                      Grill           1  2012
5         Notebook Dell 8 GB          10  2012
6            LG K10 TV Power           1  2012
7                Micro-Ondas          10  2012
8             Secadora Vapor          12  2013
9                 Fritadeira           1  2013
10                     Grill           3  2013
11                 Aspirador          18  2013
12         Forno-Micro-Ondas           5  2013
13        Impressora Deskjet          12  2013
14           Ar Condicionado           5  2013
15          Desktop HP 16 GB          12  2013
16               Micro-Ondas           4  2013
17          Geladeira Duplex          27  2013
18            Lavadora 11 Kg          12  2013
19           Sony Experia XA           1  2013
20          S

In [18]:
#2. Qual o total de venda de cada vendendor?
query= """
    SELECT nome_vendedor,count(venda_final)
    FROM vendas_final vf 
    GROUP BY nome_vendedor 
    ORDER BY count(venda_final) DESC 
"""

cursor = conn.cursor()
cursor.execute(query)

# Pegar os resultados
resultados = cursor.fetchall()

# Nome das colunas
colunas = ['nome_vendedor', 'quantidade']

# Criar o DataFrame
df_resultado = pd.DataFrame(resultados, columns=colunas)

cursor.close()
# Exibir a tabela
print(df_resultado)


       nome_vendedor  quantidade
0      André Pereira          84
1    Maria Fernandes          77
2       Josias Silva          62
3      Artur Moreira          57
4   Mateus Gonçalves          51
5   Rodrigo Fagundes          48
6       Ana Teixeira          38
7  Fernando Zambrini          30
8       Aline Sutter          10


In [20]:
#3. Percentual arrecadado por cada vendedor do total de vendas_final?
query= """
SELECT 
    nome_vendedor,
    ROUND(SUM(venda_final),2) AS total_vendedor,
    ROUND(SUM(venda_final) * 100.0 / SUM(SUM(venda_final)) OVER (),2) AS percentual_total
    FROM vendas_final
    GROUP BY nome_vendedor
    ORDER BY percentual_total DESC;
"""
cursor = conn.cursor()
cursor.execute(query)

# Pegar os resultados
resultados = cursor.fetchall()

# Nome das colunas
colunas = ['nome_vendedor', 'total_vendido', 'percentual']

# Criar o DataFrame
df_resultado = pd.DataFrame(resultados, columns=colunas)

cursor.close()
# Exibir a tabela
print(df_resultado)

       nome_vendedor total_vendido percentual
0      André Pereira     104876.14      23.82
1    Maria Fernandes      68951.26      15.66
2      Artur Moreira      61663.22      14.01
3       Josias Silva      61111.37      13.88
4   Mateus Gonçalves      44645.44      10.14
5       Ana Teixeira      35758.18       8.12
6   Rodrigo Fagundes      29869.27       6.78
7  Fernando Zambrini      25864.42       5.87
8       Aline Sutter       7514.08       1.71
